## Summary of data

This script allow to see the crimes from 2015-2020, and the judicaturas that had such cases

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# Definir paths
root = Path.cwd().parent
raw = root/'data/raw'
proc = root/'data/proc'

***
### Get list of judicaturas 

There are two sources for the judicaturas codes: File sent by Consejo, and scrapped data.

First I will load the file from Consejo. It only includes the judicaturas that substanciate penal cases

In [ ]:
cods_20 = pd.read_excel(raw/'satje/0645_Id_Judicaturas.xlsx', skiprows = 5, usecols='B:C', skipfooter=5)
cods_20 = cods_20[cods_20['ID_JUDICATURA'].str.startswith('09')].copy()
cods_20['penal'] = 'penal'
cods_20.reset_index(drop=True, inplace=True)

Get list of ready judicaturas

In [ ]:
cods_15 = list((proc/'delitos_web/').glob(r'*.xls'))
cods_15 = list(map(lambda x: x.name[8:13], cods_15))
cods_15 = pd.DataFrame(cods_15, columns=['id_judicatura'])

# Limit to only Guayaquil
cods_15 = cods_15[cods_15['id_judicatura'].str.startswith('09')].reset_index(drop=True).copy()

Merge codes to see if we get all the penal judicaturas

In [ ]:
cods = pd.merge(cods_20,
    cods_15,
    how='outer',
    left_on=['ID_JUDICATURA'],
    right_on=['id_judicatura'],
    validate='1:1',
    indicator=True)

In [36]:
cods_penal = cods.loc[cods['penal']=='penal', ['ID_JUDICATURA', 'JUDICATURA', '_merge']]
cods_penal.rename(columns={'_merge': 'descargadas'}, inplace=True)

***
### Get List of crimenes

I manually set the penalties for all the cases found in the extracted data.

In [ ]:
# Load file
causas = pd.read_excel(proc/'delitos_15_20.xlsx')

# Keep only observations that are penal
causas = causas.loc[causas['penal'].isna()]

***
### How many numbres have the secuencial

Tengo que ver cuantos # tiene el secuancial (despues de año) en cada dependencia.

Hacer un loop sobre todos los archivos. Coger las causas. Quedarnos con los numeros. Ver cuantos aparecen despues del anio.

In [45]:
# List of files
files15 = list((proc/'delitos_web/').glob('*.xls'))

In [60]:
def num_secuencial(df:pd.DataFrame):
    """
    Gets the number of secuencial for each year in df
    """
    # Delete casos not found
    df = df.loc[(~df['id_proceso'].isna())&(df['causa']!='No existe este proceso')].copy()
    df = df[['id_proceso']].copy()

    # Clean secuencial
    df['id_proceso'] = df['id_proceso'].str.replace('-', '')
    df['id_judicatura'] = df['id_proceso'].apply(lambda x: x[0:5])
    df['year'] = df['id_proceso'].apply(lambda x: x[5:9])
    df['secuencial'] = df['id_proceso'].apply(lambda x: x.split(x[5:9])[1])
    df['secuencial'] = df['secuencial'].str.replace('\D', '', regex=True)

    df['nsec'] = df['secuencial'].apply(lambda x: len(x))

    # Collapse by year
    df = df.groupby(by=['id_judicatura', 'year'], as_index=False).agg(min=('nsec', 'min'), max=('nsec', 'max'))

    return df

In [61]:
num_sec = pd.DataFrame()
for file in files15:
    df = pd.read_excel(file, dtype={'id_proceso': str})
    num_sec = pd.concat([num_sec, num_secuencial(df)], ignore_index=True)

In [69]:
# Contar cuantos anos tengo para cada judicatura
nyear = num_sec.groupby(['id_judicatura'], as_index=False).agg(nyear=('year', 'count'))
num_sec = num_sec.merge(nyear)

***
### Obtener listas de cosas a webscrapear

1. Dependencias que no se logro 2015
2. Dependencias que no teniamos en la lista
3. Sentencias que se cortaron
4. Las de 2014 hacia atras

**1. No se saco 2015**

In [80]:
# Keep only for guayas
guayas = pd.merge(cods_penal, num_sec, how='left', left_on='ID_JUDICATURA', right_on='id_judicatura', validate='1:m', indicator=True)

lista2015 = list(guayas.loc[(guayas['nyear']<=5)&(guayas['year']=='2016'), 'id_judicatura'])

In [95]:
pd.DataFrame(lista2015, columns=['id_judicatura']).to_csv(proc/'restantes_2015.csv', index=False)

**2. Dependencias no en la lista, pero penal de guayas**

In [96]:
restantesgye = list(cods_penal.loc[cods_penal['descargadas']!='both', 'ID_JUDICATURA'])
pd.DataFrame(restantesgye, columns=['id_judicatura']).to_csv(proc/'restantes_gye.csv', index=False)